<a id="title_ID"></a>
# JWST Pipeline Validation Notebook: skymatch with MIRI

<span style="color:red"> **Instruments Affected**</span>: MIRI, NIRCam 


Tested on MIRI Simulated data

### Table of Contents
<div style="text-align: left"> 

<br>  [Introduction](#intro_ID) <br> [Imports](#imports_ID) <br> [Set up association files](#associations_ID) <br> [Insert outliers](#outliers_ID) <br> [Run Pipeline](#pipeline_ID) 


</div>

<a id="intro_ID"></a>
## Introduction

This notebook processes level 2 images through the calwebb_image3 skymatch step and examines outputs for different sky method parameters.

1) Set up data path and image list file.

2) Set up association files.

3) Modify average backgrould level of input images.

4) Run skymatch step on images.

5) Run skymatch for each skymethod parameter (local, global, match, global+match). 

6) Testing other parameters (nclip, usigma, lsigma, lower, upper, skystat). 


These steps are set up with simulated MIRI F560W data of the LMC astrometric field.

The pipeline documentation can be found here: https://jwst-pipeline.readthedocs.io/en/latest/jwst/skymatch/README.html

The pipeline code is available on GitHub: https://github.com/spacetelescope/jwst

Author: T. Temim

In [1]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
data_dir = TemporaryDirectory()
os.chdir(data_dir.name)

<a id="imports_ID"></a>
### Set up import statements

The following packages are needed to run this notebook:
* jwst to run the pipeline 
* matplotlib to plot
* ci_watson to retrieve data from artifactory


In [2]:
import pytest
from astropy.io import fits
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import jwst
import json
from jwst.skymatch import skymatch
from jwst.skymatch.skyimage import SkyImage
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from jwst.associations import asn_from_list
from jwst import datamodels
from matplotlib import rcParams
from ci_watson.artifactory_helpers import get_bigdata

In [3]:
import os
os.environ['CRDS_CONTEXT']='jwst_0619.pmap'

### Print pipeline version number

In [4]:
jwst.__version__ 

'1.2.3'

### Retrieve data from Artifactory

In [5]:
input_file1 = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'outlier_detection',
                     'outlier_detection_miri_test', 
                     'det_image_seq1_MIRIMAGE_F560Wexp1_cal.fits')
input_file2 = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'outlier_detection',
                     'outlier_detection_miri_test', 
                     'det_image_seq2_MIRIMAGE_F560Wexp1_cal.fits')
input_file3 = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'outlier_detection',
                     'outlier_detection_miri_test', 
                     'det_image_seq3_MIRIMAGE_F560Wexp1_cal.fits')
input_file4 = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'outlier_detection',
                     'outlier_detection_miri_test', 
                     'det_image_seq4_MIRIMAGE_F560Wexp1_cal.fits')

In [6]:
input_files=[]
input_files=[input_file1,input_file2,input_file3,input_file4]

imlist1=['det_image_seq1_MIRIMAGE_F560Wexp1_cal.fits','det_image_seq2_MIRIMAGE_F560Wexp1_cal.fits','det_image_seq3_MIRIMAGE_F560Wexp1_cal.fits','det_image_seq4_MIRIMAGE_F560Wexp1_cal.fits']

<a id="associations_ID"></a>
### Set up association files
The level three pipeline relies on an association file to specify which files are to be combined and provide the output file name.

[Top of Page](#title_ID)

In [7]:
# use asn_from_list to create association table
cal_list=imlist1
asn = asn_from_list.asn_from_list(cal_list, rule=DMS_Level3_Base, product_name='skymatch_combined.fits')

# use this if you need to add non'science' exposure types
#asn['products'][0]['members'][1]['exptype'] = 'background'
#asn['products'][0]['members'][2]['exptype'] = 'sourcecat'

# dump association table to a .json file for use in image3
with open('skymatch_asnfile.json', 'w') as fp:
    fp.write(asn.dump()[1])

skymatch_json_file='skymatch_asnfile.json'
    
json_file = skymatch_json_file
file_list = []
file_list2 = []
with open(json_file) as json_data:
    d = json.load(json_data)
    members = d['products'][0]['members']
    for item in np.arange(0,len(members)):
        file_list.append(members[item]['expname'])
        file_list2.append(members[item]['expname'][:-5]+"_skymatch.fits")

    
asn2 = asn_from_list.asn_from_list(file_list2, rule=DMS_Level3_Base, product_name=d['products'][0]['name'])

# use this if you need to add non'science' exposure types
#asn['products'][0]['members'][1]['exptype'] = 'background'
#asn['products'][0]['members'][2]['exptype'] = 'sourcecat'

# dump association table to a .json file for use in image3
with open('skymatch_asnfile2.json', 'w') as fp:
    fp.write(asn2.dump()[1])
    
skymatch_json_file2='skymatch_asnfile2.json'

### Modify average background level of input images

In [8]:
infile01_1 = input_files[0]
infile01_2 = input_files[1]
infile02_1 = input_files[2]
infile02_2 = input_files[3]

In [9]:
img01_1 = datamodels.open(infile01_1)
img01_2 = datamodels.open(infile01_2)
img02_1 = datamodels.open(infile02_1)
img02_2 = datamodels.open(infile02_2)

In [10]:
data01_1 = img01_1.data
data01_2 = img01_2.data
data02_1 = img02_1.data
data02_2 = img02_2.data

data01_1[data01_1<=0.3]=np.nan
data01_2[data01_2<=0.3]=np.nan
data02_1[data02_1<=0.3]=np.nan
data02_2[data02_2<=0.3]=np.nan

data01_1_orig = np.copy(img01_1.data)
data01_2_orig = np.copy(img01_2.data)
data02_1_orig = np.copy(img02_1.data)
data02_2_orig = np.copy(img02_2.data)

In [11]:
# check mean values of background 
print('Mean:', np.mean(data01_1_orig[data01_1_orig<=4.0]),np.std(data01_1_orig[data01_1_orig<=4.0]))
print('Mean:', np.mean(data01_2_orig[data01_2_orig<=4.0]),np.std(data01_2_orig[data01_2_orig<=4.0]))
print('Mean:', np.mean(data02_1_orig[data02_1_orig<=4.0]),np.std(data02_1_orig[data02_1_orig<=4.0]))
print('Mean:', np.mean(data02_2_orig[data02_2_orig<=4.0]),np.std(data02_2_orig[data02_2_orig<=4.0]))

Mean: 0.93090874 0.1817127
Mean: 0.9362799 0.17500266
Mean: 0.9330347 0.17194116
Mean: 0.93216574 0.16481434


In [12]:
# creating a background image with specified mean and gaussian noise with sigma = 1.0
bkg_img_noise_neg2 = np.random.normal(-2,2*0.2,data01_1.shape)
bkg_img_noise_2 = np.random.normal(2,2*0.2,data01_1.shape)
bkg_img_noise_3 = np.random.normal(3,3*0.2,data01_1.shape)
bkg_img_noise_4 = np.random.normal(4,4*0.2,data01_1.shape)
bkg_img_noise_5 = np.random.normal(5,5*0.2,data01_1.shape)
bkg_img_noise_7 = np.random.normal(7,7*0.2,data01_1.shape)

In [13]:
# adding the new background with specified mean and gaussian noise (above) to image
img01_1.data=bkg_img_noise_2
img01_2.data=bkg_img_noise_3
img02_1.data=bkg_img_noise_5
img02_2.data=bkg_img_noise_2

In [14]:
# checking the mean and standard deviations of the new background values
print(np.nanmean(img01_1.data),np.nanstd(img01_1.data))
print(np.nanmean(img01_2.data),np.nanstd(img01_2.data))
print(np.nanmean(img02_1.data),np.nanstd(img02_1.data))
print(np.nanmean(img02_2.data),np.nanstd(img02_2.data))

1.9994819 0.39928383
3.0002592 0.60046816
4.9986267 1.0009246
1.9994819 0.39928383


In [15]:
img01_1.save(file_list2[0],overwrite=True)
img01_2.save(file_list2[1],overwrite=True)
img02_1.save(file_list2[2],overwrite=True)
img02_2.save(file_list2[3],overwrite=True)

'det_image_seq4_MIRIMAGE_F560Wexp1_cal_skymatch.fits'

<a id="pipeline_ID"></a>
## Run Pipeline steps

Run the pipeline on the data for different sets of parameters

## 1. Run skymatch step on images

In [16]:
# skymatch, local, subtract= False, all 8 images used
jwst.skymatch.skymatch_step.SkyMatchStep.call(skymatch_json_file2, skymethod='local', subtract=False,output_file='MIRI',save_results=True)

2021-07-21 09:19:46,901 - CRDS - ERROR -  Error determining best reference for 'pars-skymatchstep'  =   Unknown reference type 'pars-skymatchstep'


2021-07-21 09:19:46,904 - stpipe.SkyMatchStep - INFO - SkyMatchStep instance created.


2021-07-21 09:19:47,034 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep running with args ('skymatch_asnfile2.json',).


2021-07-21 09:19:47,036 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': '/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpafycsrcg/MIRI', 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'skymethod': 'local', 'match_down': True, 'subtract': False, 'stepsize': None, 'skystat': 'mode', 'dqbits': '0', 'lower': None, 'upper': None, 'nclip': 5, 'lsigma': 4.0, 'usigma': 4.0, 'binwidth': 0.1}


2021-07-21 09:19:47,441 - stpipe.SkyMatchStep - INFO -  


...

2021-07-21 09:19:48,055 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_1_skymatchstep.fits


2021-07-21 09:19:48,305 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_2_skymatchstep.fits


2021-07-21 09:19:48,595 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_3_skymatchstep.fits


2021-07-21 09:19:48,596 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep done


<ModelContainer>

In [17]:
img01_1_new = datamodels.open('MIRI_0_skymatchstep.fits')
img01_2_new = datamodels.open('MIRI_1_skymatchstep.fits')
img02_1_new = datamodels.open('MIRI_2_skymatchstep.fits')
img02_2_new = datamodels.open('MIRI_3_skymatchstep.fits')

print('Mean: original, new (local)')
print('Mean:',np.mean(img01_1.data),',', np.mean(img01_1_new.data))
print('Mean:',np.mean(img01_2.data),',', np.mean(img01_2_new.data))
print('Mean:',np.mean(img02_1.data),',', np.mean(img02_1_new.data))
print('Mean:',np.mean(img02_2.data),',', np.mean(img02_2_new.data))

Mean: original, new (local)
Mean: 1.9994819 , 1.9994819
Mean: 3.0002592 , 3.0002592
Mean: 4.9986267 , 4.9986267
Mean: 1.9994819 , 1.9994819


In [18]:
# skymatch, local, subtract = True, all 8 images used
jwst.skymatch.skymatch_step.SkyMatchStep.call(skymatch_json_file2, skymethod='local', subtract=True,output_file='MIRI',save_results=True)

2021-07-21 09:19:49,807 - CRDS - ERROR -  Error determining best reference for 'pars-skymatchstep'  =   Unknown reference type 'pars-skymatchstep'


2021-07-21 09:19:49,810 - stpipe.SkyMatchStep - INFO - SkyMatchStep instance created.


2021-07-21 09:19:49,937 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep running with args ('skymatch_asnfile2.json',).


2021-07-21 09:19:49,940 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': '/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpafycsrcg/MIRI', 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'skymethod': 'local', 'match_down': True, 'subtract': True, 'stepsize': None, 'skystat': 'mode', 'dqbits': '0', 'lower': None, 'upper': None, 'nclip': 5, 'lsigma': 4.0, 'usigma': 4.0, 'binwidth': 0.1}


2021-07-21 09:19:50,333 - stpipe.SkyMatchStep - INFO -  


...

2021-07-21 09:19:50,946 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_1_skymatchstep.fits


2021-07-21 09:19:51,181 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_2_skymatchstep.fits


2021-07-21 09:19:51,417 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_3_skymatchstep.fits


2021-07-21 09:19:51,418 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep done


<ModelContainer>

In [19]:
img01_1_new = datamodels.open('MIRI_0_skymatchstep.fits')
img01_2_new = datamodels.open('MIRI_1_skymatchstep.fits')
img02_1_new = datamodels.open('MIRI_2_skymatchstep.fits')
img02_2_new = datamodels.open('MIRI_3_skymatchstep.fits')

print('Mean: original, new (local)')
print('Mean:',np.mean(img01_1.data),',', np.mean(img01_1_new.data))
print('Mean:',np.mean(img01_2.data),',', np.mean(img01_2_new.data))
print('Mean:',np.mean(img02_1.data),',', np.mean(img02_1_new.data))
print('Mean:',np.mean(img02_2.data),',', np.mean(img02_2_new.data))

Mean: original, new (local)
Mean: 1.9994819 , -0.010902127
Mean: 3.0002592 , -0.003758886
Mean: 4.9986267 , 0.03006431
Mean: 1.9994819 , -0.010902127


## 2. Run skymatch for each skymethod parameter (local, global, match, global+match)

### 2.1 skymethod = 'local'

In [20]:
# skymatch, local
jwst.skymatch.skymatch_step.SkyMatchStep.call(skymatch_json_file2, skymethod='local', subtract=False,output_file='MIRI',save_results=True)

2021-07-21 09:19:52,495 - CRDS - ERROR -  Error determining best reference for 'pars-skymatchstep'  =   Unknown reference type 'pars-skymatchstep'


2021-07-21 09:19:52,498 - stpipe.SkyMatchStep - INFO - SkyMatchStep instance created.


2021-07-21 09:19:52,702 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep running with args ('skymatch_asnfile2.json',).


2021-07-21 09:19:52,705 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': '/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpafycsrcg/MIRI', 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'skymethod': 'local', 'match_down': True, 'subtract': False, 'stepsize': None, 'skystat': 'mode', 'dqbits': '0', 'lower': None, 'upper': None, 'nclip': 5, 'lsigma': 4.0, 'usigma': 4.0, 'binwidth': 0.1}


2021-07-21 09:19:53,079 - stpipe.SkyMatchStep - INFO -  


...

2021-07-21 09:19:53,617 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_1_skymatchstep.fits


2021-07-21 09:19:53,861 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_2_skymatchstep.fits


2021-07-21 09:19:54,080 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_3_skymatchstep.fits


2021-07-21 09:19:54,081 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep done


<ModelContainer>

In [21]:
img01_1_new = datamodels.open('MIRI_0_skymatchstep.fits')
img01_2_new = datamodels.open('MIRI_1_skymatchstep.fits')
img02_1_new = datamodels.open('MIRI_2_skymatchstep.fits')
img02_2_new = datamodels.open('MIRI_3_skymatchstep.fits')

print('Mean: original, new (local)')
print('Mean:',np.mean(img01_1.data),',', np.mean(img01_1_new.data))
print('Mean:',np.mean(img01_2.data),',', np.mean(img01_2_new.data))
print('Mean:',np.mean(img02_1.data),',', np.mean(img02_1_new.data))
print('Mean:',np.mean(img02_2.data),',', np.mean(img02_2_new.data))

Mean: original, new (local)
Mean: 1.9994819 , 1.9994819
Mean: 3.0002592 , 3.0002592
Mean: 4.9986267 , 4.9986267
Mean: 1.9994819 , 1.9994819


### ** Check that the original and output (new) sky values match for individual images (but not necessarily the same for all images)

##### Notes: ‘local’: compute sky background values of each input image or group of images (members of the same “exposure”). A single sky value is computed for each group of images.

### 2.2 skymethod = 'global'

In [22]:
# skymatch, global, subtract = True
jwst.skymatch.skymatch_step.SkyMatchStep.call(skymatch_json_file2, skymethod='global', subtract=True,output_file='MIRI',save_results=True)

2021-07-21 09:19:55,108 - CRDS - ERROR -  Error determining best reference for 'pars-skymatchstep'  =   Unknown reference type 'pars-skymatchstep'


2021-07-21 09:19:55,111 - stpipe.SkyMatchStep - INFO - SkyMatchStep instance created.


2021-07-21 09:19:55,279 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep running with args ('skymatch_asnfile2.json',).


2021-07-21 09:19:55,281 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': '/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpafycsrcg/MIRI', 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'skymethod': 'global', 'match_down': True, 'subtract': True, 'stepsize': None, 'skystat': 'mode', 'dqbits': '0', 'lower': None, 'upper': None, 'nclip': 5, 'lsigma': 4.0, 'usigma': 4.0, 'binwidth': 0.1}


2021-07-21 09:19:55,658 - stpipe.SkyMatchStep - INFO -  


...

2021-07-21 09:19:56,205 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_1_skymatchstep.fits


2021-07-21 09:19:56,430 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_2_skymatchstep.fits


2021-07-21 09:19:56,640 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_3_skymatchstep.fits


2021-07-21 09:19:56,642 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep done


<ModelContainer>

In [23]:
img01_1_new = datamodels.open('MIRI_0_skymatchstep.fits')
img01_2_new = datamodels.open('MIRI_1_skymatchstep.fits')
img02_1_new = datamodels.open('MIRI_2_skymatchstep.fits')
img02_2_new = datamodels.open('MIRI_3_skymatchstep.fits')

print('Mean: original, new (global)')
print('Mean:',np.mean(img01_1.data),',', np.mean(img01_1_new.data))
print('Mean:',np.mean(img01_2.data),',', np.mean(img01_2_new.data))
print('Mean:',np.mean(img02_1.data),',', np.mean(img02_1_new.data))
print('Mean:',np.mean(img02_2.data),',', np.mean(img02_2_new.data))

Mean: original, new (global)
Mean: 1.9994819 , -0.010902127
Mean: 3.0002592 , 0.98987556
Mean: 4.9986267 , 2.9882426
Mean: 1.9994819 , -0.010902127


### ** Check that the output (new) sky values are equal to the original sky value minus the lowest original sky value 

##### Notes: ‘global’: compute a common sky value for all input image and groups of images. In this setting match will compute sky values for each input image/group, find the minimum sky value, and then it will set (and/or subtract) sky value of each input image to this minimum value. This method may be useful when input images have been already matched. The 'global' method computes the minimum sky value across all input images and/or groups. That sky value is then considered to be the background in all input images.

### 2.3 skymethod = 'match', match_down=True

In [24]:
# skymatch, match down, subtract = True
jwst.skymatch.skymatch_step.SkyMatchStep.call(skymatch_json_file2, skymethod='match', match_down=True,subtract=True,output_file='MIRI',save_results=True)

2021-07-21 09:19:57,761 - CRDS - ERROR -  Error determining best reference for 'pars-skymatchstep'  =   Unknown reference type 'pars-skymatchstep'


2021-07-21 09:19:57,763 - stpipe.SkyMatchStep - INFO - SkyMatchStep instance created.


2021-07-21 09:19:57,927 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep running with args ('skymatch_asnfile2.json',).


2021-07-21 09:19:57,929 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': '/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpafycsrcg/MIRI', 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'skymethod': 'match', 'match_down': True, 'subtract': True, 'stepsize': None, 'skystat': 'mode', 'dqbits': '0', 'lower': None, 'upper': None, 'nclip': 5, 'lsigma': 4.0, 'usigma': 4.0, 'binwidth': 0.1}


2021-07-21 09:19:58,306 - stpipe.SkyMatchStep - INFO -  


...

2021-07-21 09:20:00,241 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_1_skymatchstep.fits


2021-07-21 09:20:00,469 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_2_skymatchstep.fits


2021-07-21 09:20:00,667 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_3_skymatchstep.fits


2021-07-21 09:20:00,668 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep done


<ModelContainer>

In [25]:
img01_1_new = datamodels.open('MIRI_0_skymatchstep.fits')
img01_2_new = datamodels.open('MIRI_1_skymatchstep.fits')
img02_1_new = datamodels.open('MIRI_2_skymatchstep.fits')
img02_2_new = datamodels.open('MIRI_3_skymatchstep.fits')

print('Mean: original, new (match down)')
print('Mean:',np.mean(img01_1.data),',', np.mean(img01_1_new.data))
print('Mean:',np.mean(img01_2.data),',', np.mean(img01_2_new.data))
print('Mean:',np.mean(img02_1.data),',', np.mean(img02_1_new.data))
print('Mean:',np.mean(img02_2.data),',', np.mean(img02_2_new.data))

Mean: original, new (match down)
Mean: 1.9994819 , 1.9994819
Mean: 3.0002592 , 2.0148857
Mean: 4.9986267 , 2.0459924
Mean: 1.9994819 , 1.9980334


### ** Check that all output (new) sky values are equal to the lowest original sky value 

##### Notes: ‘match’: compute differences in sky values between images and/or groups in (pair-wise) common sky regions. In this case computed sky values will be relative (delta) to the sky computed in one of the input images whose sky value will be set to (reported to be) 0. This case matches to the lowest sky value.

### 2.4 skymethod = 'match', match_down=False

In [26]:
# skymatch, match up, subtract = True
jwst.skymatch.skymatch_step.SkyMatchStep.call(skymatch_json_file2, skymethod='match', match_down=False,subtract=True,output_file='MIRI',save_results=True)

2021-07-21 09:20:01,788 - CRDS - ERROR -  Error determining best reference for 'pars-skymatchstep'  =   Unknown reference type 'pars-skymatchstep'


2021-07-21 09:20:01,790 - stpipe.SkyMatchStep - INFO - SkyMatchStep instance created.


2021-07-21 09:20:01,951 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep running with args ('skymatch_asnfile2.json',).


2021-07-21 09:20:01,954 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': '/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpafycsrcg/MIRI', 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'skymethod': 'match', 'match_down': False, 'subtract': True, 'stepsize': None, 'skystat': 'mode', 'dqbits': '0', 'lower': None, 'upper': None, 'nclip': 5, 'lsigma': 4.0, 'usigma': 4.0, 'binwidth': 0.1}


2021-07-21 09:20:02,323 - stpipe.SkyMatchStep - INFO -  


...

2021-07-21 09:20:04,226 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_1_skymatchstep.fits


2021-07-21 09:20:04,439 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_2_skymatchstep.fits


2021-07-21 09:20:04,641 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_3_skymatchstep.fits


2021-07-21 09:20:04,642 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep done


<ModelContainer>

In [27]:
img01_1_new = datamodels.open('MIRI_0_skymatchstep.fits')
img01_2_new = datamodels.open('MIRI_1_skymatchstep.fits')
img02_1_new = datamodels.open('MIRI_2_skymatchstep.fits')
img02_2_new = datamodels.open('MIRI_3_skymatchstep.fits')

print('Mean: original, new (match up)')
print('Mean:',np.mean(img01_1.data),',', np.mean(img01_1_new.data))
print('Mean:',np.mean(img01_2.data),',', np.mean(img01_2_new.data))
print('Mean:',np.mean(img02_1.data),',', np.mean(img02_1_new.data))
print('Mean:',np.mean(img02_2.data),',', np.mean(img02_2_new.data))

Mean: original, new (match up)
Mean: 1.9994819 , 4.952117
Mean: 3.0002592 , 4.9675198
Mean: 4.9986267 , 4.9986267
Mean: 1.9994819 , 4.9506664


### ** Check that all output (new) sky values are equal to the highest original sky value 

##### Notes: ‘match’: compute differences in sky values between images and/or groups in (pair-wise) common sky regions. In this case computed sky values will be relative (delta) to the sky computed in one of the input images whose sky value will be set to (reported to be) 0. This case matches to the highest sky value.

### 2.5 skymethod = 'global+match'

In [28]:
# skymatch, global+match, subtract = True
jwst.skymatch.skymatch_step.SkyMatchStep.call(skymatch_json_file2, skymethod='global+match', match_down=True,subtract=True,output_file='MIRI',save_results=True)

2021-07-21 09:20:05,764 - CRDS - ERROR -  Error determining best reference for 'pars-skymatchstep'  =   Unknown reference type 'pars-skymatchstep'


2021-07-21 09:20:05,767 - stpipe.SkyMatchStep - INFO - SkyMatchStep instance created.


2021-07-21 09:20:05,927 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep running with args ('skymatch_asnfile2.json',).


2021-07-21 09:20:05,929 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': '/data1/jenkins/workspace/Notebooks/jwst_validation_notebooks_spacetelescope/tmp/tmpafycsrcg/MIRI', 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'skymethod': 'global+match', 'match_down': True, 'subtract': True, 'stepsize': None, 'skystat': 'mode', 'dqbits': '0', 'lower': None, 'upper': None, 'nclip': 5, 'lsigma': 4.0, 'usigma': 4.0, 'binwidth': 0.1}


2021-07-21 09:20:06,304 - stpipe.SkyMatchStep - INFO -  


...

2021-07-21 09:20:08,316 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_1_skymatchstep.fits


2021-07-21 09:20:08,533 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_2_skymatchstep.fits


2021-07-21 09:20:08,743 - stpipe.SkyMatchStep - INFO - Saved model in MIRI_3_skymatchstep.fits


2021-07-21 09:20:08,744 - stpipe.SkyMatchStep - INFO - Step SkyMatchStep done


<ModelContainer>

In [29]:
img01_1_new = datamodels.open('MIRI_0_skymatchstep.fits')
img01_2_new = datamodels.open('MIRI_1_skymatchstep.fits')
img02_1_new = datamodels.open('MIRI_2_skymatchstep.fits')
img02_2_new = datamodels.open('MIRI_3_skymatchstep.fits')

print('Mean: original, new (global+match)')
print('Mean:',np.mean(img01_1.data),',', np.mean(img01_1_new.data))
print('Mean:',np.mean(img01_2.data),',', np.mean(img01_2_new.data))
print('Mean:',np.mean(img02_1.data),',', np.mean(img02_1_new.data))
print('Mean:',np.mean(img02_2.data),',', np.mean(img02_2_new.data))

Mean: original, new (global+match)
Mean: 1.9994819 , -0.009458263
Mean: 3.0002592 , 0.005945227
Mean: 4.9986267 , 0.037052132
Mean: 1.9994819 , -0.010907317


### ** Check that all output (new) sky values are around zero (within a standard deviation)

##### Notes: ‘global+match’: first use ‘match’ method to equalize sky values between images and then find a minimum “global” sky value in all input images.
